In [1]:
import numpy as np 
from math import *
import os 
import io
import pickle
import matplotlib as plt 
from PIL import Image
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
import csv
from PIL import Image
Image.MAX_IMAGE_PIXELS = 500000000

In [21]:
def onclick(event):
    if event.xdata != None and event.ydata != None :
        if event.key == " ":
            print(event.xdata, event.ydata)
            ptabcd.append([event.xdata,event.ydata])
            bcdef = plt.scatter(event.xdata, event.ydata, color = "red", s = 7)
            abcde.append(bcdef)
            plt.draw()
        if event.key == "d" and len(abcde) > 0:
            bcdef = abcde[-1]
            bcdef.remove()
            del abcde[-1]
            del ptabcd[-1]
            plt.draw()
        if event.key == "n":
            plt.close()
        
def clickfun(img):
    global ptabcd, abcde, bcdef
    ptabcd, abcde = [], []
    im = Image.open(img)
    im = im.convert("CMYK")
    ax = plt.gca()
    fig = plt.gcf()
    plt.ion()
    implot = ax.imshow(im)
    cid = fig.canvas.mpl_connect('key_press_event', onclick)
    plt.show()
    pixel = np.array(im).shape
    return [pixel, ptabcd]

def clip_photo(saveplace, photo, corpt = [0, 0, 0, 0]):
    from PIL import Image
    xcor, ycor, wlen, hlen = corpt[0], corpt[1], corpt[2], corpt[3]
    #img = Image.open(photo).convert("L")
    img = Image.open(photo)#.convert("L")
    img = np.array(img)
    #print(img)
    clipped_img = []
    for yaxis in range(hlen):
        tmp = []
        for xaxis in range(wlen):
            tmp.append(img[ycor + yaxis][xcor + xaxis])
        clipped_img.append(tmp)
    clipped_img = np.array(clipped_img)
    clipped_img = Image.fromarray(clipped_img)
    #clipped_img.show()
    clipped_img.save(saveplace)

def clip_photo_ver2(saveplace, photo, corpt = [0, 0, 0, 0]):
    from PIL import Image
    xcor, ycor, wlen, hlen = corpt[0], corpt[1], corpt[2], corpt[3]
    
    img = Image.open(photo)
    img = np.array(img)
    
    clipped_img = []
    for yaxis in range(hlen):
        tmp = []
        for xaxis in range(wlen):
            tmp.append(img[ycor + yaxis][xcor + xaxis][0]*443/1000 + img[ycor + yaxis][xcor + xaxis][1]*443/1000 + img[ycor + yaxis][xcor + xaxis][2]*114/1000)
        clipped_img.append(tmp)
    clipped_img = np.array(clipped_img)
    clipped_img = Image.fromarray(clipped_img, 'L')
    #clipped_img.show()
    clipped_img.save(saveplace)


    
def clip_3rows(photo, rows = 3):
    from PIL import Image
    import numpy as np
    img = np.array(Image.open(photo).convert('L'))
    clip = []
    rows
    for i in np.arange(0, len(img)-3, rows):
        tmp = []
        for j in range(rows):
            tmp.append(img[i+j])
        clip.append(tmp)
    tmp = []
    for k in np.arange(i, len(img), 1):
        tmp.append(img[k])
    clip.append(tmp)
            
    return clip        
            
def clip_photo_grab(ser_img, para1, para2):
    import cv2
    clip_img = []
    for i in range(len(ser_img)):
        array = np.array(ser_img[i])
        #img = cv2.GaussianBlur(ser_img[i],(3,3),0)
        canny_img = cv2.Canny(array, para1, para2) 
        for j in canny_img:
            clip_img.append(j)
    clip_img = np.array(clip_img)
    return clip_img

def photo_process(photo, para1 = 200, para2 = 220):
    target_a = clip_3rows(photo, 1)
    array = clip_photo_grab(target_a, para1, para2)
    
    #for i in np.arange(1, len(array)-1, 1):
        #for j in np.arange(1, len(array[i])-1, 1):
            #if array[i+1][j+1]+ array[i+1][j]+ array[i][j+1]+ array[i-1][j-1]+ array[i-1][j]+ array[i][j-1]+ array[i+1][j-1]+ array[i-1][j+1] <= 3:
             #if array[i+1][j]+ array[i][j+1]+ array[i-1][j]+ array[i][j-1] <=1:
                #array[i][j] = 0
    #Image.fromarray(array, 'L').show()
    '''with open("/Users/pengbohao/Downloads/edge detection/testdouble03.pickle", 'wb') as f:
        pickle.dump(array, f)'''
    return array

def double2coordinate(array):
    coordinate = []
    for i in range(len(array)):
        for j in range(len(array[i])):
            if array[i][j] != 0:
                coordinate.append([j, i])
    return coordinate

def Out_of_blockade(list_pt, convolution = [3, 5], step = 3):
    def fliter(rail, n, convol):
        count = sum([rail.count(n - int(convol[0]/2)) for i in range(convol[0])])
        return count
    rail = [i[0] for i in list_pt]
    height, slope, gradi2 = [], [], []
    X = 0
    while X + int(convolution[0]/2) <= max(rail):
        height.append(fliter(rail, X, convolution))
        X = X + step
    return height

def list_blockade(coordinate, deviation = 0.3, size = [11, 25], step = 10):
    classi = [[] for i in range(max(np.array(coordinate).T[1]))]
    result = []
    for i in range(len(coordinate)):
        classi[coordinate[i][1]-1].append(coordinate[i])
    
    # try blockade
    for i in np.arange(0, len(classi) - size[1], size[1]):
        tar = []
        for j in range(size[1]):
            tar = tar + classi[i+j]
        result.append(Out_of_blockade(tar, convolution = size, step = step))
    
    # record restruct
    record = []
    for i in range(len(result)):
        for j in range(len(result[i])-1):
            if abs(result[i][j+1] - result[i][j])/size[0]/size[1] >= deviation:
                record.append([j*step, i*size[1]])
    return record

def record_blockade(photo, p1, p2, deviation, size, step):
    array = photo_process(photo, para1 = p1, para2 = p2)
    coordinate = double2coordinate(array)
    record = list_blockade(coordinate, deviation, size = size, step = step)
    return record


def find_bothside_point(record, tagpoint, deviation, firstpoint_u = [100, 200], firstpoint_d = [100, 200]):
    x, yu, yd = sum(np.array(tagpoint).T[0])/len(tagpoint), min(np.array(tagpoint).T[1]), max(np.array(tagpoint).T[1])
    y_list = sorted(list(set(np.array(record).T[1])))
    level_list = [[] for i in range(len(y_list))]
    for i in range(len(record)):
        level_list[y_list.index(record[i][1])].append(record[i])

    # down 
    X = firstpoint_d
    Y = int(yd)+ 50
    d_leftpoint, d_rightpoint, u_leftpoint, u_rightpoint = [], [], [], []
    last_dev = 0

    while Y < max(np.array(record).T[1]):
        if Y in y_list:
            group = level_list[y_list.index(Y)]
            r_index = sorted(np.array(group).T[0].tolist() + [X[1]]).index(X[1])
            l_index = sorted(np.array(group).T[0].tolist() + [X[0]]).index(X[0]) - 1
            if r_index == len(group):
                pass
            else:
                if len(d_rightpoint) != 0:
                    while sorted(group)[r_index] != sorted(group)[-1] and abs((sorted(group)[r_index][0] - d_rightpoint[-1][0])/ (sorted(group)[r_index][1] - d_rightpoint[-1][1])) > deviation:
                        r_index = r_index - 1
                    devi = abs((sorted(group)[r_index][0] - d_rightpoint[-1][0])/ (sorted(group)[r_index][1] - d_rightpoint[-1][1])) 
                    if abs(devi - last_dev) < deviation:
                        d_rightpoint.append(sorted(group)[r_index])
                        last_dev = devi
                else:
                    d_rightpoint.append(sorted(group)[r_index])
            if l_index == -1:
                pass
            else:
                if len(d_leftpoint) != 0:
                    while sorted(group)[l_index] != sorted(group)[0] and abs((sorted(group)[l_index][0] - d_leftpoint[-1][0])/ (sorted(group)[l_index][1] - d_leftpoint[-1][1])) > deviation:
                        l_index = l_index - 1
                    devi = abs((sorted(group)[l_index][0] - d_leftpoint[-1][0])/ (sorted(group)[l_index][1] - d_leftpoint[-1][1]))
                    if abs(devi - last_dev) < deviation:
                        d_leftpoint.append(sorted(group)[l_index])
                        last_dev = devi  
                else:
                    d_leftpoint.append(sorted(group)[l_index])
        Y = Y + 1
        if len(d_leftpoint) != 0 and len(d_rightpoint) != 0:
            X = [(d_leftpoint[-1][0] + d_rightpoint[-1][0])/2, (d_leftpoint[-1][0] + d_rightpoint[-1][0])/2]
            
    # up
    X = firstpoint_d
    Y = int(yu)-2
    last_dev = 0
    while Y > min(np.array(record).T[1]):
        if Y in y_list:
            group = level_list[y_list.index(Y)]
            r_index = sorted(np.array(group).T[0].tolist() + [X[1]]).index(X[1])
            l_index = sorted(np.array(group).T[0].tolist() + [X[0]]).index(X[0]) - 1
            if r_index == len(group):
                pass
            else:
                if len(u_rightpoint) != 0:
                    while sorted(group)[r_index] != sorted(group)[-1] and abs((sorted(group)[r_index][0] - u_rightpoint[-1][0])/ (sorted(group)[r_index][1] - u_rightpoint[-1][1])) > deviation:
                        r_index = r_index - 1 
                    devi = abs((sorted(group)[r_index][0] - u_rightpoint[-1][0])/ (sorted(group)[r_index][1] - u_rightpoint[-1][1]))
                    if abs(devi - last_dev) < deviation:
                        u_rightpoint.append(sorted(group)[r_index])
                        last_dev = devi
                else:
                    u_rightpoint.append(sorted(group)[r_index])
            if l_index == -1:
                pass
            else:
                if len(u_leftpoint) != 0:
                    while sorted(group)[l_index] != sorted(group)[0] and abs((sorted(group)[l_index][0] - u_leftpoint[-1][0])/ (sorted(group)[l_index][1] - u_leftpoint[-1][1])) > deviation:
                        l_index = l_index - 1
                    devi = abs((sorted(group)[l_index][0] - u_leftpoint[-1][0])/ (sorted(group)[l_index][1] - u_leftpoint[-1][1]))
                    if abs(devi - last_dev) < deviation:
                        u_leftpoint.append(sorted(group)[l_index])
                        last_dev = devi  
                else:
                    u_leftpoint.append(sorted(group)[l_index])
        
        Y = Y-1
        
        if len(u_leftpoint) != 0 and len(u_rightpoint) != 0:
            X = [(u_leftpoint[-1][0] + u_rightpoint[-1][0])/2, (u_leftpoint[-1][0] + u_rightpoint[-1][0])/2]
    return u_leftpoint, u_rightpoint, d_leftpoint, d_rightpoint
    
    
    
def tree_contour(photo, series_pt, devi, direct = 'u'):
    im = Image.open(photo)
    plt.imshow(im)
    def cal_dev(cor1, cor2):
        a = cor2[0] - cor1[0]
        b = cor2[1] - cor1[1]
        return a/b
    
    series_pt = sorted(list(series_pt), key = lambda x: x[1])
    '''if direct == 'u' or 'U':
        ptset = []
        dev = cal_dev(series_pt[0], series_pt[1])
        i = 1
        while i != len(series_pt) - 1:
            if abs(cal_dev(series_pt[i-1], series_pt[i]) - dev) > devi:
                del series_pt[i]
            else:
                ptset.append(series_pt[i])
                i+=1
        series_pt = ptset
    
    else:
        ptset = []
        series_pt.reverse()
        dev = cal_dev(series_pt[0], series_pt[1])
        i = 1
        while i != len(series_pt) - 1:
            if abs(cal_dev(series_pt[i-1], series_pt[i]) - dev) > devi:
                del series_pt[i]
            else:
                ptset.append(series_pt[i])
                i+=1
        series_pt = ptset '''
        
    for n in np.arange(0, len(series_pt)-1, 1):
        x = [series_pt[n][0], series_pt[n+1][0]]
        y = [series_pt[n][1], series_pt[n+1][1]]
        plt.plot(x, y)
        plt.draw()
    '''for n in np.arange(0, len(series_pt)-2, 2):
        series_pt[n], series_pt[n+1], series_pt[n+2]
        x = [series_pt[n][0], series_pt[n+1][0], series_pt[n+2][0]]
        y = [series_pt[n][1], series_pt[n+1][1], series_pt[n+2][1]]
        p = np.poly1d(np.polyfit(y, x, 2))
        Y = np.linspace(series_pt[n][1], series_pt[n+2][1])
        plt.plot(p(Y), Y)
        plt.draw()'''
    plt.show()
    

#tree_contour('/Users/pengbohao/Downloads/edge detection/01.jpg', u_leftpoint, 2)
#tree_contour('/Users/pengbohao/Downloads/edge detection/01.jpg', u_rightpoint, 2)
#tree_contour('/Users/pengbohao/Downloads/edge detection/01.jpg', d_leftpoint, 2)
#tree_contour('/Users/pengbohao/Downloads/edge detection/01.jpg', d_rightpoint, 2)


def draw_contour(photo, record, tagpoint, deviation2, firstpoint_u, firstpoint_d):
    #record = record_blockade(photo, p1, p2, deviation1, size, step)
    ul, ur, dl, dr = find_bothside_point(record, tagpoint, deviation2, firstpoint_u, firstpoint_d)
    ulp, urp, dlp, drp = ul[0], ur[0], dl[0], dr[0]
    im = Image.open(photo)
    plt.imshow(im)
    def cal_dev(cor1, cor2):
        a = cor2[0] - cor1[0]
        b = cor2[1] - cor1[1]
        return a/b
    points = [ul, ur, dl, dr]
    for point in points:
        for n in np.arange(0, len(point)-1, 1):
            x = [point[n][0], point[n+1][0]]
            y = [point[n][1], point[n+1][1]]
            plt.plot(x, y, color = 'red')
            plt.draw()
    plt.show()



In [650]:
def density2Height(Coordinate, Number, Rows):
    import numpy as np
    All_Height = []
    for i in np.arange(0, len(Coordinate)-1, Rows):
        Height = []
        Target = np.array(Coordinate[i:i+Rows]).T.tolist()
        for j in range(len(Target)):
            Height.append(sum(Target[j])/255)
        All_Height.append(Height)
    return All_Height
        
def Height2Slope(All_Height):
    import numpy as np
    All_Slope, All_Acceleration = [], []
    for i in range(len(All_Height)):
        Slope, Acceleration = [], []
        for j in range(len(All_Height[i]) - 1):
            Slope.append(All_Height[i][j + 1] - All_Height[i][j])
        for j in range(len(All_Height[i]) - 2):
            front = All_Height[i][j + 2] - All_Height[i][j + 1]
            behind = All_Height[i][j + 1] - All_Height[i][j]
            Acceleration.append(front - behind)
        All_Slope.append(Slope)
        All_Acceleration.append(Acceleration)
    return All_Slope, All_Acceleration
    
def count_point(All_Slope, All_Acceleration, Rows):
    Slope_points, Acceler_points = [], [] 
    for i in range(len(All_Slope)):
        for j in range(len(All_Slope[i])-1):
            if abs(All_Slope[i][j]) >= Rows - 1 and abs(All_Slope[i][j]) != abs(All_Slope[i][j+1]):
                Slope_points.append([j, i*Rows])
                
    for i in range(len(All_Acceleration)):
        for j in range(len(All_Acceleration[i])):
            if All_Acceleration[i][j] <= -Rows:
                Acceler_points.append([j, i*Rows])
                
    return Slope_points, Acceler_points

def Find_block(set_points, initialpoint, deviation, Rows, width):
    import numpy as np 
    UIPL, UIPR = initialpoint[0], initialpoint[1]
    DIPL, DIPR = initialpoint[2], initialpoint[3]
    #len(set(np.array(set_points).T.tolist()[1]))
    range_points = [[] for i in range(len(set(np.array(set_points).T.tolist()[1])))]
    set_points = sorted(set_points, key = lambda x : x[1])
    n = 0
    range_points[n].append(set_points[0])
    Din, Uin =  10000000000, 0
    for i in np.arange(1, len(set_points)):
        if set_points[i][1] == set_points[i - 1][1]:
            if set_points[i][1] >= max([DIPL[1], DIPR[1]]):
                range_points[n].append(set_points[i])
                Din = min([Din, n])
            elif set_points[i][1] <= min([UIPL[1], UIPR[1]]):
                range_points[n].append(set_points[i])
                Uin = max([Uin, n])
        else:
            n += 1
            if set_points[i][1] >= max([DIPL[1], DIPR[1]]):
                range_points[n].append(set_points[i])
                Din = min([Din, n])
            elif set_points[i][1] <= min([UIPL[1], UIPR[1]]):
                range_points[n].append(set_points[i])
                Uin = max([Uin, n])
               
    num = 1       
    UMX, DMX = (UIPL[0]+UIPR[0])/2, (DIPL[0]+DIPR[0])/2
    Border_points_L, Border_points_R = [[UIPL[0], UIPL[1]]], [[UIPR[0], UIPR[1]]]
    Ldev, Rdev =  (UIPL[0] - DIPL[0])/(UIPL[1] - DIPL[1]), (UIPR[0] - DIPR[0])/(UIPR[1] - DIPR[1])
    RightPoint, LeftPoint = 0, 0
    for j in np.arange(Uin, 0, -1):
        M = (sorted((np.array(range_points[j]).T.tolist()[0]) +[UMX])).index(UMX)
        M_index_right, M_index_left = M, M-1
        while M_index_left > 0:
            a = (range_points[j][M_index_left][0] - Border_points_L[-1][0])
            b = (range_points[j][M_index_left][1] - Border_points_L[-1][1])
            if b == 0:
                b = num
            if ((a/b) - Ldev) > deviation:
                M_index_left = 0
            elif ((a/b) - Ldev) < (-1)*deviation:
                M_index_left = M_index_left - 1 
            else:
                LeftPoint = (range_points[j][M_index_left])
                M_index_left = 0
                #Ldev = a/b
                
        while M_index_right < len(range_points[j]) - 1:    
            c = (range_points[j][M_index_right][0] - Border_points_R[-1][0])
            d = (range_points[j][M_index_right][1] - Border_points_R[-1][1])
            if d == 0:
                d = num
            if ((c/d) - Rdev) > deviation:
                M_index_right = len(range_points[j]) - 1
            elif ((c/d) - Rdev) < (-1)*deviation:
                M_index_right += 1
            else:    
                RightPoint = (range_points[j][M_index_right])
                M_index_right = len(range_points[j]) - 1
                #Rdev = (c/d)
                
        if RightPoint != 0:
            e = abs(Border_points_R[-1][0] - UMX)
            if abs(RightPoint[0] - UMX) <= width*e and abs(RightPoint[0] - UMX) >= (2 - width)*e:
                Border_points_R.append(RightPoint)
                RightPoint = 0
        if LeftPoint != 0:
            f = abs(UMX - Border_points_L[-1][0])
            if abs(UMX - LeftPoint[0]) <= width*f and abs(UMX - LeftPoint[0]) >= (2 - width)*f:
                Border_points_L.append(LeftPoint)
                LeftPoint = 0
        Ldev = a/b
        Rdev = (c/d)
        UMX = (Border_points_L[-1][0] + Border_points_R[-1][0])/2      
              
    Border_points_L.append([DIPL[0], DIPL[1]])
    Border_points_R.append([DIPR[0], DIPR[1]])
    Ldev, Rdev =  (UIPL[0] - DIPL[0])/(UIPL[1] - DIPL[1]), (UIPR[0] - DIPR[0])/(UIPR[1] - DIPR[1])
    for j in np.arange(Din, len(range_points), 1):
        M = (sorted((np.array(range_points[j]).T.tolist()[0]) +[DMX])).index(DMX)
        M_index_right, M_index_left = M, M-1
        while M_index_left > 0:
            a = (range_points[j][M_index_left][0] - Border_points_L[-1][0])
            b = (range_points[j][M_index_left][1] - Border_points_L[-1][1])
            if b <= 5:
                b = num
            if ((a/b) - Ldev) > deviation:
                M_index_left = 0
            elif ((a/b) - Ldev) < (-1)*deviation:
                M_index_left = M_index_left - 1 
            else:
                LeftPoint = (range_points[j][M_index_left])
                M_index_left = 0
                #Ldev = a/b
        while M_index_right < len(range_points[j]) - 1:    
            c = (range_points[j][M_index_right][0] - Border_points_R[-1][0])
            d = (range_points[j][M_index_right][1] - Border_points_R[-1][1])
            if d == 0:
                d = num
            if ((c/d) - Rdev) > deviation:
                M_index_right = len(range_points[j]) - 1
            elif ((c/d) - Rdev) < (-1)*deviation:
                M_index_right += 1
            else:    
                RightPoint = (range_points[j][M_index_right])
                M_index_right = len(range_points[j]) - 1
                #Rdev = (c/d)
        
        if RightPoint != 0:
            e = abs(Border_points_R[-1][0] - DMX)
            if abs(RightPoint[0] - DMX) <= width*e and abs(RightPoint[0] - DMX) > (2 - width)*e:
                Border_points_R.append(RightPoint)
                RightPoint = 0
        if LeftPoint != 0:
            f = abs(DMX - Border_points_L[-1][0])
            if abs(DMX - LeftPoint[0]) <= width*f and abs(DMX - LeftPoint[0]) >= (2 - width)*f:
                Border_points_L.append(LeftPoint)
                LeftPoint = 0

        Ldev = a/b
        Rdev = (c/d)
        DMX = (Border_points_L[-1][0] + Border_points_R[-1][0])/2
    #Border_points_L + Border_points_R            
    return Border_points_L + Border_points_R

def Find_block_width(set_points, initialpoint, width):
    import numpy as np 
    UIPL, UIPR = initialpoint[0], initialpoint[1]
    DIPL, DIPR = initialpoint[2], initialpoint[3]
    #len(set(np.array(set_points).T.tolist()[1]))
    range_points = [[] for i in range(len(set(np.array(set_points).T.tolist()[1])))]
    set_points = sorted(set_points, key = lambda x : x[1])
    n = 0
    range_points[n].append(set_points[0])
    Din, Uin =  10000000000, 0
    for i in np.arange(1, len(set_points)):
        if set_points[i][1] == set_points[i - 1][1]:
            if set_points[i][1] >= max([DIPL[1], DIPR[1]]):
                range_points[n].append(set_points[i])
                Din = min([Din, n])
            elif set_points[i][1] <= min([UIPL[1], UIPR[1]]):
                range_points[n].append(set_points[i])
                Uin = max([Uin, n])
        else:
            n += 1
            if set_points[i][1] >= max([DIPL[1], DIPR[1]]):
                range_points[n].append(set_points[i])
                Din = min([Din, n])
            elif set_points[i][1] <= min([UIPL[1], UIPR[1]]):
                range_points[n].append(set_points[i])
                Uin = max([Uin, n])
    num = 1       
    UMX, DMX = (UIPL[0]+UIPR[0])/2, (DIPL[0]+DIPR[0])/2
    Border_points_L, Border_points_R = [[UIPL[0], UIPL[1]]], [[UIPR[0], UIPR[1]]]
    
    LW = abs(Border_points_L[-1][0] - UMX)
    RW = abs(Border_points_R[-1][0] - UMX)
    for j in np.arange(Uin, 0, -1):
        tmpL, tmpR = [], []
        for k in range(len(range_points[j])):
            target = UMX - range_points[j][k][0]
            if target > 0: #L
                if target >= (2 - width)**2*LW and target <= width*LW:
                    tmpL.append(range_points[j][k])
            else: #R
                if abs(target) >= (2 - width)**2*RW and abs(target) <= width*RW: 
                    tmpR.append(range_points[j][k])
        if len(tmpR) != 0:
            RX = sum(np.array(tmpR).T.tolist()[0])/len(tmpR)
            #RX = np.median(np.array(tmpR).T[0])
        else:
            RX = Border_points_R[-1][0]
        if len(tmpL) != 0:
            LX = sum(np.array(tmpL).T.tolist()[0])/len(tmpL)
            #LX = np.median(np.array(tmpL).T[0])
        else:
            LX = Border_points_L[-1][0]
        Border_points_L, Border_points_R = Border_points_L+tmpL, Border_points_R+tmpR
        if len(tmpR) != 0 and len(tmpL) != 0:
            UMX = np.mean(np.array(np.array(tmpR).T.tolist()[0]+np.array(tmpL).T.tolist()[0]))
        else:
            UMX = (RX + LX)/2    
        LW, RW = abs(LX - UMX), abs(RX - UMX)
    Border_points_L.append([DIPL[0], DIPL[1]])
    Border_points_R.append([DIPR[0], DIPR[1]])      
    LW = abs(Border_points_L[-1][0] - DMX)
    RW = abs(Border_points_R[-1][0] - DMX)
    for j in np.arange(Din, len(range_points), 1):    
        tmpL, tmpR = [], []
        for k in range(len(range_points[j])):
            target = DMX - range_points[j][k][0]
            if target > 0: #L
                if target >= (2 - width)**2*LW and target <= width*LW:
                    tmpL.append(range_points[j][k])
            else: #R
                if abs(target) >= (2 - width)**2*RW and abs(target) <= width*RW: 
                    tmpR.append(range_points[j][k])
        if len(tmpR) != 0:
            RX = sum(np.array(tmpR).T.tolist()[0])/len(tmpR)
            #RX = np.median(np.array(tmpR).T[0])
        else:
            RX = Border_points_R[-1][0]
        if len(tmpL) != 0:
            LX = sum(np.array(tmpL).T.tolist()[0])/len(tmpL)
            #LX = np.median(np.array(tmpL).T[0])
        else:
            LX = Border_points_L[-1][0]
        Border_points_L, Border_points_R = Border_points_L+tmpL, Border_points_R+tmpR
        if len(tmpR) != 0 and len(tmpL) != 0:
            DMX = np.mean(np.array(np.array(tmpR).T.tolist()[0]+np.array(tmpL).T.tolist()[0]))
        else:
            DMX = (RX + LX)/2
        LW, RW = abs(LX - DMX), abs(RX - DMX)
        
        
    return Border_points_L + Border_points_R



In [52]:
plt.ion()